In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# 1. Data Cleaning & Pre-Processing & Initial Data Transformation

## 1.1. Table - "Sales"

a) Import table "sales.xlsx"

In [30]:
sales = pd.read_excel('sales.xlsx')
#sales.set_index('CustomerID', inplace = True)
sales

,CustomerID,Date_Adherence,Recency,MntMeat&Fish,MntEntries,MntVegan&Vegetarian,MntDrinks,MntDesserts,MntAdditionalRequests,NumOfferPurchases,NumAppPurchases,NumTakeAwayPurchases,NumStorePurchases,NumAppVisitsMonth,Complain
0,11054,2021-03-18 00:00:00,85.0,7230.0,1820,3435.0,1600.0,440.0,57.0,1,13,4,13,6,0
1,17117,2021-03-01 00:00:00,96.0,5850.0,320,2745.0,500.0,320.0,8.0,3,9,7,9,6,0
2,31751,2021-11-01 00:00:00,7.0,8380.0,0,1560.0,0.0,0.0,28.0,3,9,5,3,9,0
3,12947,2022-05-19 00:00:00,54.0,5090.0,280,2355.0,140.0,560.0,78.0,3,7,5,11,3,0
4,11402,2021-01-01 00:00:00,64.0,330.0,0,240.0,0.0,0.0,7.0,2,4,1,3,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,23699,2021-11-27 00:00:00,54.0,290.0,20,180.0,0.0,40.0,4.0,2,4,1,4,7,0
7004,23255,2021-03-25 00:00:00,27.0,230.0,80,525.0,20.0,80.0,23.0,3,5,1,3,8,0
7005,5798,2022-04-12 00:00:00,36.0,3050.0,2460,3840.0,1800.0,980.0,107.0,1,8,4,13,4,0
7006,16577,2020-09-01 00:00:00,49.0,20.0,200,90.0,160.0,0.0,14.0,1,3,2,2,8,1


b) Check for duplicates (and drop)

In [48]:
#sales [sales.duplicated()]
sales.drop_duplicates(inplace= True)

c) Check for NAs & Fill NAs

In [32]:
#sales.isna().sum()

## Input "Recency" with the mean
sales.fillna({'Recency': sales['Recency'].mean()}, inplace = True)
#Turn "Recency" into integer
sales['Recency']= np.round(sales['Recency']).astype(int)

## Inpute "MntDrinks" using KNNInputer
#we are using this variables to inpute values because they have a correlation > 0.7
sales_products = sales[['MntDrinks','MntDesserts','MntEntries','MntVegan&Vegetarian']]

imputer = KNNImputer(n_neighbors=1)
array_impute = imputer.fit_transform(sales_products) # this is an array
sales_products = pd.DataFrame(array_impute, columns = sales_products.columns)

sales['MntDrinks'] = sales_products['MntDrinks'].values

## 1.2. Demographic Table - "CRM"

### 1.2.1. Clean Data 

d) Import table "crm.xlsx"

In [33]:
crm = pd.read_csv('crm.csv')
crm.set_index('CustomerID', inplace = True)
crm

,Name,Birthyear,Education,Marital_Status,Income,Kid_Younger6,Children_6to18
CustomerID,,,,,,,
14594,Miss Leah Lewis,1981,Graduation,Single,113947.0,0,0
19070,Miss Grace Black,1974,Graduation,Together,114297.0,0,0
29888,Mr. Gavin Morrison,1960,PhD,Together,115476.0,0,0
12221,Mr. Ryan Glover,1980,Graduation,Married,101172.0,1,1
21107,Mr. Paul Anderson,1971,HighSchool,Single,123128.0,0,0
...,...,...,...,...,...,...,...
30686,Mr. John Hughes,1994,PhD,Divorced,98968.0,0,0
17210,Mr. Brandon Simpson,1985,Graduation,Divorced,109977.0,0,0
29186,Mr. Thomas Churchill,1967,PhD,Together,56039.0,0,1


e) Check for duplicates (and drop)

In [54]:
#crm[crm.duplicated()]

f) Check for NAs & Fill NAs

In [35]:
#crm.isnull().sum()

## Substitute missing values with the mode
crm['Education'] = crm['Education'].fillna(crm['Education'].mode()[0])

g) Consistency accross the variables/categories

In [36]:
## Putting everything upper case
crm['Education'] = crm['Education'].str.upper()
crm['Marital_Status']=crm['Marital_Status'].str.upper()

## Change together to married 
crm['Marital_Status'] = crm['Marital_Status'].replace('TOGETHER','MARRIED')
#crm['Marital_Status'].value_counts()

### 1.2.2. Add useful variables (data transformation)

h) Add column with gender - dummy variable - easier to read

In [42]:
## Check for all unique prefixes, we extract the first 4 letters of each name

crm['First 4 Letters'] = crm['Name'].str.slice(stop=4)

# get the unique combinations of the first 5 letters
unique_combinations = crm['First 4 Letters'].unique()

# print the unique combinations
print(unique_combinations)

crm['Name'].isnull().any()

#Now, we do the encoding. If the first 4 characters contain any variation of mrs or miss, we substitute the name by 1
#if it's any variation of 'mr.' we substitute by 0

# create a copy of the 'Name' column
crm['Gender'] = crm['Name']

# replace 'Miss' or 'Mrs.' with 1 and 'Mr.' with 0 in the 'Gender' column
crm.loc[crm['Gender'].str.contains(r'^((M|m)iss|(M|m)rs\.)', na=False), 'Gender'] = 1
crm.loc[crm['Gender'].str.contains(r'^(Mr\.|(M|m)r)', na=False), 'Gender'] = 0

# convert the 'Gender' column to integer type
crm['Gender'] = crm['Gender'].astype(float).fillna(0).astype(int)

#check new column
crm['Gender']

crm.dropna(subset=['First 4 Letters'])

['Miss' 'Mr. ' 'Mrs.']


,Name,Birthyear,Education,Marital_Status,Income,Kid_Younger6,Children_6to18,First 4 Letters,Gender
CustomerID,,,,,,,,,
14594,Miss Leah Lewis,1981,GRADUATION,SINGLE,113947.0,0,0,Miss,1
19070,Miss Grace Black,1974,GRADUATION,MARRIED,114297.0,0,0,Miss,1
29888,Mr. Gavin Morrison,1960,PHD,MARRIED,115476.0,0,0,Mr.,0
12221,Mr. Ryan Glover,1980,GRADUATION,MARRIED,101172.0,1,1,Mr.,0
21107,Mr. Paul Anderson,1971,HIGHSCHOOL,SINGLE,123128.0,0,0,Mr.,0
...,...,...,...,...,...,...,...,...,...
30686,Mr. John Hughes,1994,PHD,DIVORCED,98968.0,0,0,Mr.,0
17210,Mr. Brandon Simpson,1985,GRADUATION,DIVORCED,109977.0,0,0,Mr.,0
29186,Mr. Thomas Churchill,1967,PHD,MARRIED,56039.0,0,1,Mr.,0


i) Transform "Birthyear" into "Age"

In [43]:
## Convert the "Birthyear" to an 'Age' column
import datetime

#calculate the current year
current_year = datetime.datetime.now().year

#create a new column 'Age' by subtracting birth_year from current year
crm['Age'] = current_year - crm['Birthyear']

# drop
crm = crm.drop('Birthyear', axis=1)


## 1.3. Marketing Table - "mkt.xlsx"

### 1.3.1. Data Cleaning

j) Import table "mkt.xlsx"

In [46]:
mkt = pd.read_csv('mkt.csv')
mkt[mkt.duplicated(subset= 'CustomerID', keep=False)]
mkt

,CustomerID,Response_Cmp1,Response_Cmp2,Response_Cmp3,Response_Cmp4,Response_Cmp5
0,31958,0,0,0,0,0
1,5387,0,0,0,0,0
2,32006,0,0,0,0,0
3,7511,0,0,0,0,0
4,12308,0,0,0,0,0
...,...,...,...,...,...,...
7018,22358,0,0,0,0,0
7019,29516,0,0,0,0,0
7020,18890,0,0,0,0,0
7021,30206,0,0,0,0,0


k) Check for duplicates (and drop)

In [52]:
#mkt[mkt.duplicated()]

l) Check for NAs & Fill NAs

In [53]:
#mkt.isna().sum()

# 2. Outliers

# 3. Data Exploration 

# 4. 1st Perspective - Consumption patterns/types

EXPLICAÇÃO DA PERSPECTIVA:
- Objetivos;
- Interesse;
- variáveis foco;

## 4.1. Dataframe Preparation

- 1- Isolar variáveis de interesse
- 2- Standardização (MinMax)
- 3- PCA???!!!

## 4.2. K-means

## 4.3. DBSCAN

## 4.4. Cluster analysis & caracterization

## 4.5. Business & Marketing insights

# 5. 2nd Perspective - Preference in the means of purchase

EXPLICAÇÃO DA PERSPECTIVA:

 - Objetivos;
 - Interesse;
 - variáveis foco;

## 5.1. Dataframe Preparation

## 5.2. K-means

## 5.3. DBSCAN

## 5.4. Fuzzy K-means (?)

## 5.5. Cluster analysis & caracterization

## 5.6. Business & Marketing insights

# 6. 3rd Perspective - Consumer value

EXPLICAÇÃO DA PERSPECTIVA:

- Objetivos;
- Interesse;
- variáveis foco;

## 6.1. Dataframe Preparation

## 6.2. K-means

## 6.3. Cluster analysis & caracterization

## 6.4. Business & Marketing insights